In [2]:
import logging
import requests
from io import BytesIO
from pdfminer.high_level import extract_text


def download_pdf(url: str) -> BytesIO:
    """
    指定された URL から PDF をダウンロードし、BytesIO オブジェクトとして返す。
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # HTTP エラーがあれば例外を発生させる
        return BytesIO(response.content)
    except requests.exceptions.RequestException as e:
        logging.error("PDF のダウンロード中にエラーが発生しました: %s", e)
        raise


def extract_text_from_pdf(pdf_file: BytesIO) -> str:
    """
    BytesIO 形式の PDF ファイルからテキストを抽出する。
    """
    try:
        text = extract_text(pdf_file)
        return text
    except Exception as e:
        logging.error("PDF からのテキスト抽出中にエラーが発生しました: %s", e)
        raise

In [ ]:
url = "https://www.fastretailing.com/jp/ir/library/pdf/20250109_results.pdf"

# PDF をダウンロード
pdf_file = download_pdf(url)

# PDF からテキストを抽出
text = extract_text_from_pdf(pdf_file)

# 結果を標準出力に表示
print(text)

In [1]:
import logging
import os
import requests
import tempfile
from markitdown import MarkItDown


def download_pdf(url: str, dest_path: str) -> None:
    """
    指定された URL から PDF をダウンロードし、dest_path に保存する。
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # HTTP エラーがあれば例外を発生させる
        with open(dest_path, "wb") as f:
            f.write(response.content)
    except requests.exceptions.RequestException as e:
        logging.error("PDF のダウンロード中にエラーが発生しました: %s", e)
        raise


def convert_pdf_to_markdown(pdf_path: str) -> str:
    """
    markitdown を使用して、指定された PDF ファイルから Markdown 形式のテキストを抽出する。
    """
    try:
        md = MarkItDown()
        result = md.convert(pdf_path)
        return result.text_content
    except Exception as e:
        logging.error("PDF からの Markdown 変換中にエラーが発生しました: %s", e)
        raise

In [3]:
temp_pdf_path

'/tmp/tmp0nqp4rpn.pdf'

In [2]:
# 例: URL から PDF をダウンロードし、Markdown に変換する
url = "https://www.fastretailing.com/jp/ir/library/pdf/20250109_results.pdf"
with tempfile.NamedTemporaryFile(suffix=".pdf", delete=False) as tmp:
    temp_pdf_path = tmp.name

try:
    download_pdf(url, temp_pdf_path)
    markdown_text = convert_pdf_to_markdown(temp_pdf_path)
    print(markdown_text)
finally:
    if os.path.exists(temp_pdf_path):
        os.remove(temp_pdf_path)

2025年１月９日

2025年８月期 第１四半期業績
および通期 業績予想

岡﨑 健
株式会社ファーストリテイリング
取締役 グループ上席執行役員 CFO

１

CFOの岡﨑です。

私から、2025年８月期第１四半期の業績、およ(cid:23773)
2025年８月期の業績予想について、ご説明いたします。

目次

Ⅰ．第１四半期 決算概要
Ⅱ．2025年８月期 業績予想
Ⅲ．参考資料

P ３
P 23
P 28

～ P 22
  ～ P 27
  ～ P 31

【業績開示について】
・2014年８月期末より国際財務報告基準（IFRS）を適用、本資料上の数字は、すべてIFRSベースで記載しております。
・事業利益は、売上収益から売上原価、販管費を控除して算出しております。
・各セグメントの構成は、以下の通りです。

国内ユニクロ事業：日本で展開するユニクロ事業が含まれております。
海外ユニクロ事業：海外で展開するユニクロ事業が含まれております。
ジーユー事業：日本・海外で展開するジーユー事業が含まれております。
グローバルブランド事業：セオリー事業、プラステ事業、コントワー・デ・コトニエ事業、プリンセス タム・タム事業が含まれております。

・連結業績には上記のほか、ファーストリテイリングの業績、連結調整などが含まれております。

【将来予測に関するご注意】
・本資料に掲載されている業績予想、計画、目標数値などのうち、歴史的事実でないものは、作成時点で入手可能な情報に基づき作成した将来情報です。
実際の業績は、経済環境、市場の需要、価格競争に対する対応、為替などの変動により、この業績予想、計画、目標数値と大きく異なる場合があります。

2

【連結】2025年８月期 第１四半期 実績

大幅な増収増益。営業利益はほぼ業績予想通り

3

2025年８月期第１四半期の連結業績ですが、
売上収益は8,951億円、前年同期比10.4％増、
事業利益は1,569億円、同11.0%増、
営業利益は1,575億円、同7.4％の増となりました。

10月に発表した業績予想に対しては、
売上収益は業績予想を若干下回る水準となりましたが、
営業利益はほぼ業績予想通りの水準となっております。

次のページで第１四半期の決算ハイライトをお伝えいたします。

　単位：億円